# Overlap Correction with Linear Models (aka unfold.jl)
### Benedikt Ehinger, with help Dave Kleinschmidt
### 2020-02-17

In [ ]:
using Revise
using CSV
using StatsModels
using MixedModels
using DataFrames
using DataFramesMeta
using Gadfly
import DSP
import Plots
import unfold
Plots.gr()

In this notebook we will fit regression models to (simulated) EEG data. We will see that we need some type of overlap correction, as external events are so close to each other, that the respective brain responses overlap.
If you want more detailed introduction to this topic check out my paper: https://peerj.com/articles/7838/

In [ ]:
testCase = "testCase2"
#testCase = "testCaseMultisubject"
data = CSV.read("test\\"*testCase*"_data.csv", header=0)
data = convert(Matrix,data)
data = dropdims(data,dims=1) # convert to vector
evts = CSV.read("test\\"*testCase*"_events.csv");

In [ ]:
show(first(evts,6,),allcols=true)

The data has little noise and the underlying signal is a pos-neg spike pattern

In [ ]:
Plots.plot(data[1:300])

## Traditional Mass Univariate Analysis
In order to demonstrate why overlap correction is important, we will first epoch the data and fit a linear model to each time point.
This is a "traditional mass-univariate analysis".

In [ ]:
# for future multi-channel support (not yet there!)
data_r = reshape(data,(1,:))
# cut the data into epochs
data_epochs,times = unfold.epoch(data=data_r,tbl=evts,τ=(-0.4,0.8),sfreq=50)

We define a formula that we want to apply to each point in time

In [ ]:
f  = @formula 0~1+conditionA*conditionB

We fit the special-type `UnfoldLinearModel` to the data

In [ ]:
m = unfold.fit(unfold.UnfoldLinearModel,f,evts,data_epochs,times)

The object has the following fields

In [ ]:
println(typeof(m))
println(fieldnames(typeof(m)))

Which contain the model, the original formula, the original events and also a *tidy*-dataframe with the results

In [ ]:
first(m.results,6)

We can also plot it using Gadfly

In [ ]:
d = @linq m.results |> where(:group.=="mass univariate")
plot(d,x=:time,y=:estimate,color=:term,Geom.LineGeometry)

## Basis Functions
#### HRF / BOLD
We are now ready to define a basisfunction. There are currently only two basisfunction implemented, so not much choice.
We first have a look at the BOLD-HRF basisfunction:

In [ ]:
TR = 1.5
bold = unfold.hrfbasis(TR) # using default SPM parameters
eventonset = 1.3
Plots.plot(bold.kernel(eventonset))

Classically, we would convolve this HRF function with a impulse-vector, with impulse at the event onsets

In [ ]:
y = zeros(100)
y[[10,30,37,45]] .=1
y_conv = DSP.conv(y,bold.kernel(0))
Plots.plot(y_conv)

Which one would use as a regressor against the recorded BOLD timecourse.

Note that events could fall inbetween TR (the sampling rate). Some packages subsample the time signal, but in `unfold` we can directly call the `bold.kernel` function at a given event-time, which allows for non-TR-multiples to be used.

### FIR Basis Function

Okay, let's have a look at a different basis function: The FIR basisfunction.

In [ ]:
basisfunction = unfold.firbasis(τ=(-0.4,.8),sfreq=50)
Plots.plot(basisfunction.kernel(0))

Not very clear, better show it in 2D

In [ ]:
basisfunction.kernel(0)[1:10,1:10]

The FIR basisset consists of multiple basisfunctions. That is, each event will now be *timeexpanded* to multiple predictors, each with a certain time-delay to the event onset.
This allows to model any arbitrary linear overlap shape, and doesn't force us to make assumptions on the convolution kernel (like we had to do in the BOLD case)


## Timeexpanded / Deconvolved ModelFit
We can now define the formula again

In [ ]:
f  = @formula 0~1+conditionA*conditionB

For the left-handside we use "0" as the data is separated from the events. This will in the future allow us to fit multiple channels easily.

And fit a `UnfoldLinearModel`. Not that instead of `times` as in the mass-univariate case, we have a `BasisFunction` object now.

In [ ]:
m = unfold.fit(unfold.UnfoldLinearModel,f,evts,data,basisfunction)

In [ ]:
#Gadfly.push_theme(:dark)
d = @linq m.results |> where(:group.=="fixed")
plot(d,x=:time,y=:estimate,color=:term,Geom.LineGeometry)

In [ ]:
using Weave

# convert to html
weave("doc\\lm_tutorial.jmd")

# convert to a python notebook
convert_doc("doc\\lm_tutorial.jmd", "doc\\lm_tutorial.ipynb")

# convert to md for README
weave("doc\\lm_tutorial.jmd", doctype="pandoc", out_path = "README.md")